In [8]:
import pandas as pd
import numpy as np

# Đọc dữ liệu sạch
graduate = pd.read_csv('data_clean/graduate_info_clean.csv')
employment = pd.read_csv('data_clean/employment_survey_clean.csv')
certifications = pd.read_csv('data_clean/certifications_clean.csv')

print("Dữ liệu đã được tải thành công!")

Dữ liệu đã được tải thành công!


# 1. Tạo pivot thể hiện mức lương khởi điểm trung bình theo ngành × năm tốt nghiệp (major × grad_year). 

In [9]:
print("\n" + "="*60)
print("Mức lương khởi điểm trung bình theo ngành × năm tốt nghiệp")
print("="*60)

salary_df = graduate.merge(
    employment,
    on='student_id',
    how='inner'
)

# Pivot lương – CHỈ LẤY MEAN
pivot_salary = pd.pivot_table(
    salary_df,
    values='salary',
    index='major',
    columns='grad_year',
    aggfunc='mean'
)

print("Pivot lương trung bình:")
print(pivot_salary.round(0))

# Tìm ngành – năm lương cao nhất
max_value = pivot_salary.max().max()
best_position = pivot_salary.stack().idxmax()

print("\nKẾT LUẬN:")
print(
    f"Ngành có mức lương cao nhất là: '{best_position[0]}'\n"
    f"Năm: {best_position[1]}\n"
    f"Mức lương trung bình: {max_value:,.0f} VNĐ."
)



Mức lương khởi điểm trung bình theo ngành × năm tốt nghiệp
Pivot lương trung bình:
grad_year                  2021       2022       2023
major                                                
Công nghệ thông tin  10000000.0  9166667.0  9000000.0
Kinh tế               9333333.0  9000000.0  8666667.0
Quản trị kinh doanh   9000000.0  8800000.0  9166667.0

KẾT LUẬN:
Ngành có mức lương cao nhất là: 'Công nghệ thông tin'
Năm: 2021
Mức lương trung bình: 10,000,000 VNĐ.


# 2.Tạo pivot thể hiện số chứng chỉ trung bình theo ngành × grad_year.

In [10]:
print("\n" + "="*50)
print("Số chứng chỉ trung bình theo ngành × grad_year")
print("="*50)

# Đếm số chứng chỉ mỗi sinh viên
cert_count = certifications.groupby('student_id').size().reset_index(name='cert_count')
cert_df = graduate.merge(
    cert_count,
    on='student_id',
    how='left'
)
cert_df['cert_count'] = cert_df['cert_count'].fillna(0)  #Điền 0 cho người không có chứng chỉ
#Pivot chứng chỉ TB
pivot_cert = pd.pivot_table(
    cert_df,
    values='cert_count',
    index='major',
    columns='grad_year',
    aggfunc='mean'
)
print("Pivot số chứng chỉ trung bình:\n")
print(pivot_cert)




Số chứng chỉ trung bình theo ngành × grad_year
Pivot số chứng chỉ trung bình:

grad_year                2021      2022      2023
major                                            
Công nghệ thông tin  0.857143  0.857143  1.571429
Kinh tế              1.300000  1.000000  0.833333
Quản trị kinh doanh  2.000000  0.846154  1.277778


# 3.Stack và unstack để chuyển đổi bảng pivot sang dạng MultiIndex

In [11]:
#Chuyển đổi DataFrame sang Series đa cấp
salary_series = pivot_salary.stack()
print("="*55)
print("Stack (Series đa cấp):")
print("="*55)
print(salary_series.round(0))

#Chuyển đổi Series đa cấp về DataFrame  
salary_df_back = salary_series.unstack()
print("\n" + "="*55)
print("Sau unstack (DataFrame):")
print("="*55)
print(salary_df_back.round(0))

Stack (Series đa cấp):
major                grad_year
Công nghệ thông tin  2021         10000000.0
                     2022          9166667.0
                     2023          9000000.0
Kinh tế              2021          9333333.0
                     2022          9000000.0
                     2023          8666667.0
Quản trị kinh doanh  2021          9000000.0
                     2022          8800000.0
                     2023          9166667.0
dtype: float64

Sau unstack (DataFrame):
grad_year                  2021       2022       2023
major                                                
Công nghệ thông tin  10000000.0  9166667.0  9000000.0
Kinh tế               9333333.0  9000000.0  8666667.0
Quản trị kinh doanh   9000000.0  8800000.0  9166667.0


# Phân tích sự thay đổi theo ngành và năm tốt nghiệp 

In [12]:
# Phân tích xu hướng lương theo năm
salary_change = pivot_salary.diff(axis=1)
print("="*60)
print("Mức thay đổi lương theo năm (tăng giảm):")
print("="*60 + "\n")
print(salary_change)

# Ngành & năm có mức lương TỐT NHẤT
max_salary = pivot_salary.max().max()
best = pivot_salary.stack().idxmax()

print(f"\nNgành có lương cao nhất: {best[0]}")
print(f"Năm: {int(best[1])}")
print(f"Lương TB: {max_salary:,.0f}")

# Số NaN theo ngành (hàng)
print("\n" + "="*60)
print("Số NaN theo ngành (hàng):")
print(pivot_salary.isna().sum(axis=1))

# Số NaN theo năm (cột)
print("\n" + "="*60)
print("Số NaN theo năm (cột):")
print(pivot_salary.isna().sum(axis=0))


Mức thay đổi lương theo năm (tăng giảm):

grad_year            2021           2022           2023
major                                                  
Công nghệ thông tin   NaN -833333.333333 -166666.666667
Kinh tế               NaN -333333.333333 -333333.333333
Quản trị kinh doanh   NaN -200000.000000  366666.666667

Ngành có lương cao nhất: Công nghệ thông tin
Năm: 2021
Lương TB: 10,000,000

Số NaN theo ngành (hàng):
major
Công nghệ thông tin    0
Kinh tế                0
Quản trị kinh doanh    0
dtype: int64

Số NaN theo năm (cột):
grad_year
2021    0
2022    0
2023    0
dtype: int64
